# Transform Longitude Values

Many climate datasets have longitude coordinates from 0 to 360. This is not suitable to be used in other raster based geoprocessing systems. This notebook shows how to convert such datasets to span longitude form -180 to +180.

We take a NetCDF file of Monthly Precipitation by [ECMWF ERA5-Land Monthly Average](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means) dataset. This file has global monthly precipitation for the month of July, 2021. We reproject the longitude coordinates and save the result as a GeoTIFF file.

Input:
* `era5-land-precipitation.nc`: A NetCDF file containing monthly average precipitation with longitude coordinates from 0-360

Output:
* `precipitation.tif`: A reprojected GeoTiff file of total monthly precpitation with longitudes from -180 to +180


Data Credit:

Muñoz Sabater, J., (2019): ERA5-Land monthly averaged data from 1981 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). (Accessed on < 05-Aug-2022 >), 10.24381/cds.68d2bb3

## Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [93]:
if 'google.colab' in str(get_ipython()):
    !pip install rioxarray --quiet

In [94]:
import os
import xarray as xr
import rioxarray as rxr
import pandas as pd 

In [95]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [112]:
file = 'era5-land-precipitation.nc'
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/spatialthoughts/python-tutorials/raw/main/data/' + file)

## Procedure

In [113]:
input_path = os.path.join(data_folder, file)

Open the input file.

In [115]:
da = xr.open_dataset(input_path)

Notice the `longitude` coordinates have have values from 0 to 360.

In [99]:
da

<xarray.Dataset>
Dimensions:    (longitude: 3600, latitude: 1801, time: 1)
Coordinates:
  * longitude  (longitude) float32 0.0 0.1 0.2 0.3 ... 359.6 359.7 359.8 359.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 2021-07-01
Data variables:
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-08-05 17:45:55 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

We now convert the longitude to range from -180 to +180.

In [100]:
da.coords['longitude'] = (da.coords['longitude'] + 180) % 360 - 180
da = da.sortby(da.longitude)

Notice the `longitude` coordinates have have values from -180 to +180

In [116]:
da

<xarray.Dataset>
Dimensions:    (longitude: 3600, latitude: 1801, time: 1)
Coordinates:
  * longitude  (longitude) float32 0.0 0.1 0.2 0.3 ... 359.6 359.7 359.8 359.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 2021-07-01
Data variables:
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-08-05 17:45:55 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

The dataset is now ready to be saved. But before saving, we can also convert the units to be more usable. The input dataset's units are **meters/day**, so we multiply by 1000 and the number of days in the month to get total precipitation in **mm/month**. [[reference](https://confluence.ecmwf.int/pages/viewpage.action?pageId=197702790)]

We take the `time` value and calculate the number of days in the month using `pd.Period`.

In [102]:
t = da.time.values[0]
ts = pd.to_datetime(t).strftime('%Y-%m-%d')
days = pd.Period(ts).days_in_month
print('timestamp', ts)
print('number of days', days)

timestamp 2021-07-01
number of days 31


In [109]:
total = da*1000*days

Rasterio can only save 2D arrays, so we can just select the first time band

In [110]:
output = total.isel(time=0)
output

<xarray.Dataset>
Dimensions:    (longitude: 3600, latitude: 1801)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
    time       datetime64[ns] 2021-07-01
Data variables:
    tp         (latitude, longitude) float32 nan nan nan ... 7.841 7.841 7.841
Attributes:
    Conventions:  CF-1.6
    history:      2022-08-05 17:45:55 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [111]:
output_file = 'precipitation.tif'.format(ts)
output_path = os.path.join(output_folder, output_file)
output.rio.to_raster(output_path)